In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import median_absolute_error, accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/user/conda/lib/python3.9/site-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


In [3]:
# utils

def map_class(x, task, reader):
    if task.name == 'multiclass':
        return reader[x]
    else:
        return x

mapped = np.vectorize(map_class)

def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return accuracy_score(y_true, np.argmax(y_pred, 1))
    elif task.name == 'reg' or task.name == 'multi:reg':
        return median_absolute_error(y_true, y_pred)
    else:
        raise 'Task is not correct.'
        
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'
        
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

### 0.2 Constants

Here we setup the constants to use in the kernel:
- `N_THREADS` - number of vCPUs for LightAutoML model creation
- `N_FOLDS` - number of folds in LightAutoML inner CV
- `RANDOM_STATE` - random seed for better reproducibility
- `TEST_SIZE` - houldout data part size 
- `TIMEOUT` - limit in seconds for model to train
- `TARGET_NAME` - target column name in dataset
- `TASK` - task name, 'reg', 'binary', 'multiclass', 'multi:reg'
- `ALGOS_FOR_BLEND` - algorithms used in blending
- `USE_PLR` - if True use PLR embedder for continuous features, else Basic Embedder
- `TRAIN_BS` - train batch size

In [4]:
RANDOM_STATE = 42
N_THREADS = 16

In [5]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

### 0.3 Data loading

In [6]:
train = pd.read_csv('../input/playground-series-s4e7/train.csv')
test = pd.read_csv('../input/playground-series-s4e7/test.csv')

In [7]:
#X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)

In [8]:
task = Task('binary') #‘binary’ 
automl = TabularAutoML(
    task = task, 
    timeout = 600 * 3600,
    cpu_limit = 16,
    general_params = {"use_algos": [['fttransformer']]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {
        "n_epochs": 10, 
        "bs": 1024, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": 'plr',
        'cat_embedder': 'weighted',
        "hidden_size": 32,
        'embedding_size': 32,
        'verbose_bar': True,
        "snap_params": { 'k': 1, 'early_stopping': True, 'patience': 1, 'swa': True }
    },
    nn_pipeline_params = {"use_qnt": False, "use_te": False},
    reader_params = {'n_jobs': 16, 'cv': 5, 'random_state': 42, 'advanced_roles': True}
)

out_of_fold_predictions = automl.fit_predict(
    train, #valid_data=X_val,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

[20:57:01] Stdout logging level is DEBUG.
[20:57:01] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[20:57:01] Task: binary

[20:57:01] Start automl preset with listed constraints:
[20:57:01] - time: 2160000.00 seconds
[20:57:01] - CPU: 16 cores
[20:57:01] - memory: 16 GB

[20:57:01] Train data shape: (11504798, 12)

[20:57:29] Feats was rejected during automatic roles guess: []
[20:57:30] Layer 1 train process start. Time left 2159971.48 secs
[20:57:56] number of text features: 0 
[20:57:56] number of categorical features: 8 
[20:57:56] number of continuous features: 2 
[20:57:56] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_fttransformer_0 ...
[20:57:56] Training params: {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': device(type='cuda', index=0), 'use_cont': True, 'use_cat': True, 'use_text': False, 'lang': 'en', 'determi

val: 100%|██████████| 2248/2248 [00:37<00:00, 60.01it/s]


[21:01:55] Epoch: 0, train loss: 0.24999429285526276, val loss: 0.24415138363838196, val metric: 0.8907484089947391


val: 100%|██████████| 2248/2248 [00:38<00:00, 57.65it/s]


[21:05:58] Epoch: 1, train loss: 0.24259915947914124, val loss: 0.24316208064556122, val metric: 0.8917082937767429


...

[22:01:30] Epoch: 1, train loss: 0.24265095591545105, val loss: 0.24293029308319092, val metric: 0.8919983503258939


val: 100%|██████████| 2248/2248 [00:38<00:00, 58.80it/s]


[22:05:32] Epoch: 2, train loss: 0.24032406508922577, val loss: 0.2434309870004654, val metric: 0.8919275685130066


val: 100%|██████████| 2248/2248 [00:39<00:00, 57.40it/s]


[22:06:12] Early stopping: val loss: 0.24293029308319092, val metric: 0.8919983503258939
[22:06:17] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_fttransformer_0 finished. score = 0.8915023652738223
[22:06:17] Lvl_0_Pipe_0_Mod_0_TorchNN_fttransformer_0 fitting and predicting completed
[22:06:17] Time left 2155844.03 secs

[22:06:17] Layer 1 training completed.

[22:06:17] Automl preset training completed in 4156.06 seconds

[22:06:17] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_fttransformer_0) 



In [9]:
roc_auc_score(train.Response, out_of_fold_predictions.data)

0.8915023652738223

In [10]:
pred = automl.predict(test)

test: 100%|██████████| 7491/7491 [01:11<00:00, 104.14it/s]


In [11]:
import joblib
joblib.dump((out_of_fold_predictions.data[:, 0], pred.data[:, 0]), 'fttransformer_5fold_oof_test_089150.jbl')
joblib.dump(automl, 'fttransformer_5fold_model_089150.jbl')

['fttransformer_5fold_model_089150.jbl']